Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [2]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [3]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [4]:
import numpy as np 

def wrangle(X):

  #prevent SettingWithCopy warning
  X = X.copy()

  #replace really small nums in latitude with zeros
  X['latitude'] = X['latitude'].replace(-2.000000e-08, 0)

  #for both cols replace zeros with nans:
  cols_with_zeros = ['longitude', 'latitude', 'construction_year', 'public_meeting', ]
  for col in cols_with_zeros:
    X[col] = X[col].replace(0, np.nan)

  X['date_recorded'] = pd.to_datetime(X['date_recorded'])
  X['day_recorded'] = pd.DatetimeIndex(X['date_recorded']).day
  X['month_recorded'] = pd.DatetimeIndex(X['date_recorded']).month
  X['year_recorded'] = pd.DatetimeIndex(X['date_recorded']).year
  X['years_since_construction'] = X['year_recorded'] - X['construction_year']

  # Reduce cardinality for funder feature ...
  top49 = train['funder'].value_counts()[:49].index
  X.loc[~X['funder'].isin(top49), 'funder'] = 'other'

  #enough water?
  X['enough'] = np.where(X['quantity'] == 'enough', 1, 0)


  #peple per water
  X['pop_per_water'] = X['amount_tsh'] / X['population']

  #drop one of the same/almost same cols, cols with a lot of missing data
  X = X.drop(columns= ['quantity_group', 'date_recorded', 'payment_type', 'num_private',  
                       'source_class', 'source_type', 'waterpoint_type_group', 'quality_group',
                       'extraction_type_group', 'extraction_type_class', 'management_group', 'scheme_management'])

  #return wrangled dataset
  return X

In [5]:
train = wrangle(train)
test = wrangle(test)

In [6]:
#select features
target = 'status_group'

#get a data frame with all features
train_features = train.drop(columns = [target,'id', 'recorded_by'])

#get numeric features
numeric = train_features.select_dtypes(include='number').columns.tolist()

#get cardinality of categorical colummns
cardinality = train_features.select_dtypes(exclude = 'number').nunique()

#exlude columns with cardinality more than 50 
categorical = cardinality[cardinality <= 50 ].index.tolist()

#combine
features = categorical + numeric

print(features)

['funder', 'basin', 'region', 'public_meeting', 'permit', 'extraction_type', 'management', 'payment', 'water_quality', 'quantity', 'source', 'waterpoint_type', 'amount_tsh', 'gps_height', 'longitude', 'latitude', 'region_code', 'district_code', 'population', 'construction_year', 'day_recorded', 'month_recorded', 'year_recorded', 'years_since_construction', 'enough', 'pop_per_water']


In [7]:
X_train = train[features]
y_train = train[target]

In [8]:
X_train['quantity'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64

In [9]:
y_train

0            functional
1            functional
2            functional
3        non functional
4            functional
              ...      
59395        functional
59396        functional
59397        functional
59398        functional
59399        functional
Name: status_group, Length: 59400, dtype: object

In [10]:
X_train.head()

,funder,basin,region,public_meeting,permit,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,amount_tsh,gps_height,longitude,latitude,region_code,district_code,population,construction_year,day_recorded,month_recorded,year_recorded,years_since_construction,enough,pop_per_water
0,Roman,Lake Nyasa,Iringa,True,False,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,6000.0,1390,34.938093,-9.856322,11,5,109,1999.0,14,3,2011,12.0,1,55.045872
1,other,Lake Victoria,Mara,NaN,True,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,0.0,1399,34.698766,-2.147466,20,2,280,2010.0,6,3,2013,3.0,0,0.000000
2,other,Pangani,Manyara,True,True,gravity,vwc,pay per bucket,soft,enough,dam,communal standpipe multiple,25.0,686,37.460664,-3.821329,21,4,250,2009.0,25,2,2013,4.0,1,0.100000
3,Unicef,Ruvuma / Southern Coast,Mtwara,True,True,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,0.0,263,38.486161,-11.155298,90,63,58,1986.0,28,1,2013,27.0,0,0.000000
4,other,Lake Victoria,Kagera,True,True,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,0.0,0,31.130847,-1.825359,18,1,0,NaN,13,7,2011,NaN,0,NaN


In [11]:
X_train = X_train.replace([np.inf, -np.inf], np.nan)

In [12]:
# imports
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import cross_val_score

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    #StandardScaler(),
    #SelectKBest(f_regression, k = 40),
    RandomForestClassifier(random_state=0, n_jobs=-1)
)

k = 5 
scores = cross_val_score(pipeline, X_train, y_train, cv=k, 
                         scoring='accuracy')
print(f'Accuracy for {k} folds:', scores)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Accuracy for 5 folds: [0.81212121 0.80479798 0.80681818 0.80387205 0.80260943]


In [13]:
print('Model Hyperparameters:')
print(pipeline.named_steps['randomforestclassifier'])
print(pipeline.named_steps['ordinalencoder'])
print(pipeline.named_steps['simpleimputer'])

Model Hyperparameters:
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)
OrdinalEncoder(cols=None, drop_invariant=False, handle_missing='value',
               handle_unknown='value', mapping=None, return_df=True, verbose=0)
SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)


In [14]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

param_distributions = {
    #'simpleimputer__strategy' : ['mean', 'median'], 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, 30, 40, None],
    'randomforestclassifier__n_estimators': randint(50, 500),
    'randomforestclassifier__max_features': uniform(0, 1),
    'randomforestclassifier__min_samples_leaf': [10,20,30,40,50],
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions= param_distributions,
    n_iter = 10,
    cv = 5,
    scoring = 'accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   39.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 12.2min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 17.1min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 23.0min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 24.2min finished


In [15]:
print('Best hyperparameters: ', search.best_params_)
print('Best accuracy: ', search.best_score_)

Best hyperparameters:  {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.23666788577357034, 'randomforestclassifier__min_samples_leaf': 10, 'randomforestclassifier__n_estimators': 254}
Best accuracy:  0.7966161616161616


In [16]:
pipeline = search.best_estimator_

In [18]:
X_test = test[features]

In [20]:
X_test = X_test.replace([np.inf, -np.inf], np.nan)

In [21]:
y_pred = pipeline.predict(X_test)

In [22]:
print(len(test))
print(len(y_pred))

14358
14358


In [23]:
test.index

RangeIndex(start=0, stop=14358, step=1)

In [24]:
sample_submission.head()

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,functional
4,49871,functional


In [25]:
submission=pd.DataFrame(y_pred, columns=['status_group'])
submission['id']=sample_submission.id
submission=submission[['id', 'status_group']]
submission.tail()

,id,status_group
14353,39307,non functional
14354,18990,functional
14355,28749,functional
14356,33492,functional
14357,68707,non functional


In [26]:
submission.to_csv('submission_file.csv',index=False)